In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import numpy as np

In [2]:
#Getting all columns and rows from the file
ds1_train = pd.read_csv('dataset_1/training_set.csv')
ds1_val = pd.read_csv('dataset_1/validation_set.csv')
ds1_test = pd.read_csv('dataset_1/test_set.csv')

ds2_train = pd.read_csv('dataset_2/training_set.csv')
ds2_val = pd.read_csv('dataset_2/validation_set.csv')
ds2_test = pd.read_csv('dataset_2/test_set.csv')

In [3]:
#DataSet 1
ds1_train_X = ds1_train.iloc[ 0: , 0: len( ds1_train.columns ) - 1  ].values
ds1_train_Y = ds1_train.iloc[ 0: , len( ds1_train.columns ) - 1: len( ds1_train.columns ) ].values

ds1_val_X = ds1_val.iloc[ 0: , 0: len( ds1_val.columns ) - 1  ].values
ds1_val_Y = ds1_val.iloc[ 0: , len( ds1_val.columns ) - 1: len( ds1_val.columns ) ].values

ds1_test_X = ds1_test.iloc[ 0: , 0: len( ds1_test.columns ) - 1  ].values
ds1_test_Y = ds1_test.iloc[ 0: , len( ds1_test.columns ) - 1: len( ds1_test.columns ) ].values

#DataSet 2
ds2_train_X = ds2_train.iloc[ 0: , 0: len( ds2_train.columns ) - 1  ].values
ds2_train_Y = ds2_train.iloc[ 0: , len( ds2_train.columns ) - 1: len( ds2_train.columns ) ].values

ds2_val_X = ds2_val.iloc[ 0: , 0: len( ds2_val.columns ) - 1  ].values
ds2_val_Y = ds2_val.iloc[ 0: , len( ds2_val.columns ) - 1: len( ds2_val.columns ) ].values

ds2_test_X = ds2_test.iloc[ 0: , 0: len( ds2_test.columns ) - 1  ].values
ds2_test_Y = ds2_test.iloc[ 0: , len( ds2_test.columns ) - 1: len( ds2_test.columns ) ].values

In [4]:
#------- Some "Helper" functions ---------------
# Segregating out instances that take a particular value
# attributearray is an N x 1 array.
def segregate(attributearray, value):
    outlist = []
    for i in range ( len(attributearray) ):
        if attributearray[i] == value:
            outlist.append(i)  #  Append "i" to outlist
    return outlist

In [5]:
# Assuming labels take values 1..M.
def computeEntropy(labels):
    entropy = 0
    UniqueValuesInLabels =  np.unique( labels )
    for i in UniqueValuesInLabels:
        probability_i = len(segregate(labels, i)) / len(labels)
        entropy -= probability_i * math.log2(probability_i)
    return entropy

In [6]:
# Find most frequent value. Assuming labels take values 1..M 
def mostFrequentlyOccurringValue(labels):
    bestCount = -math.inf
    bestId = None
    UniqueValuesInLabels =  np.unique( labels )
    for i in UniqueValuesInLabels:
        count_i = len(segregate(labels,i))
        if count_i > bestCount:
            bestCount = count_i
            bestId = i
    return bestId

In [7]:
def computeVarianceImpurity(labels):
    vi = 1
    UniqueValuesInLabels =  np.unique( labels )
    for i in UniqueValuesInLabels:
        probability_i = len(segregate(labels, i)) / len(labels)
        vi *= probability_i
    return vi

In [8]:
def InformationGainByVI( S, X ):
    vi = computeVarianceImpurity( S )
    UniqueValuesofX = np.unique(X)
    for Y in UniqueValuesofX:
        ids = segregate( X, Y )
        pr_x = len(ids) / len(S)
        vi_Sx = computeVarianceImpurity( S[ids] )
        vi -= pr_x * vi_Sx
    return vi

In [9]:
class DecisionTree:
    def __init__(self, attributes, labels, methodToUse ):
        self.parent = None
        self.nodeGainRation = float(0.0)
        self.nodeInformationGain = float(0.0)
        self.isLeaf = False
        self.majorityClass = int(0)
        self.bestAttribute = int(0)
        self.children = {}
        
        if methodToUse == "GainRatio":
            self.BuildTreeByGainRatio( attributes, labels )
        if methodToUse == "VarianceImpurity":
            self.BuildTreeByVI( attributes, labels )
            
    def BuildTreeByVI( self, attributes, labels ): 
        numInstances = len( labels )
        nodeInformation = numInstances * computeVarianceImpurity(labels)
        self.majorityClass = mostFrequentlyOccurringValue( labels )
        
        if nodeInformation == 0:
            self.isLeaf = True
            return
        
        bestAttribute = None
        bestInformationGain = -math.inf
        loop = 0
        for X in attributes.T:
            infoGain = InformationGainByVI(labels,X)                  
            
            if infoGain > bestInformationGain:
                bestInformationGain = infoGain
                bestAttribute = loop
            loop += 1

        if bestInformationGain == -math.inf or bestInformationGain == 0:
            self.isLeaf = True
            return 
        
        # Otherwise split by the best attribute
        self.bestAttribute = bestAttribute
        self.nodeInformationGain = bestInformationGain
        UniqueValuesInX =  np.unique( attributes[:,bestAttribute] )
        for Y in UniqueValuesInX:
            ids = segregate(attributes[:,bestAttribute], Y)
            self.children[Y] =  DecisionTree(attributes[ids], labels[ids], "VarianceImpurity")
            self.children[Y].parent = self
        return
        
                                            
    def BuildTreeByGainRatio( self, attributes, labels ):
        numInstances = len( labels )
        nodeInformation = numInstances * computeEntropy(labels)
        self.majorityClass = mostFrequentlyOccurringValue( labels )
        
        if nodeInformation == 0:
            self.isLeaf = True
            return
        
        bestAttribute = None
        bestInformationGain = -math.inf
        bestGainRatio = -math.inf
        loop = 0
        for X in attributes.T:
            conditionalInfo = 0
            attributeEntropy = 0
            attributeCount = {}
            UniqueValuesInX =  np.unique( X )
            for Y in UniqueValuesInX:
                ids = segregate( X, Y )
                attributeCount[Y] = len(ids)
                conditionalInfo += len(ids) * computeEntropy(labels[ids])

                attributeInformationGain =  nodeInformation - conditionalInfo
            attributeCount = np.array(list(attributeCount.items()), dtype=type(attributeCount))
            
            if computeEntropy(attributeCount[:,1]) != 0:
                gainRatio = attributeInformationGain / computeEntropy(attributeCount[:,1])
                if gainRatio > bestGainRatio:
                    bestInformationGain = attributeInformationGain
                    bestGainRatio = gainRatio
                    bestAttribute = loop
            loop+=1
        
        #If no attribute provides andy gain, this node cannot be split further
        if bestGainRatio == 0 or bestGainRatio == -math.inf:
            self.isLeaf = True
            return
        
        # Otherwise split by the best attribute
        self.bestAttribute = bestAttribute
        self.nodeGainRatio = bestGainRatio
        self.nodeInformationGain = bestInformationGain
        UniqueValuesInX =  np.unique( attributes[:,bestAttribute] )
        for Y in UniqueValuesInX:
            ids = segregate(attributes[:,bestAttribute], Y)
            self.children[Y] =  DecisionTree(attributes[ids], labels[ids], "GainRatio")
            self.children[Y].parent = self
        return

    def RemoveChild(self, twig):
        if self == twig:
            self.chidren = None  # Kill the chilren
            self.isLeaf = True
            self.nodeInformationGain = 0
            return  
        else:
            for val,child in self.children.items():
                child.RemoveChild( twig )

    #print (nodeInformation)
    def Evaluate(self, testAttributes):
        #print(testAttributes)
        if (self.isLeaf):
            return self.majorityClass
        else:
            return self.children[
                testAttributes[self.bestAttribute]
            ].Evaluate(testAttributes)
        
    def PrintRec(self, columnName, ans, bestAttribute):
        if (self.isLeaf):
            ans.append( str( self.majorityClass ) ) 
            return ans
        else:
            
            ans.append ( str( columnName[bestAttribute] ) + " = 0 : " )
            self.children[0].PrintRec( columnName, ans, self.children[0].bestAttribute )
            
            ans.append ( str( columnName[bestAttribute] ) + " = 1 : " )
            self.children[1].PrintRec( columnName, ans, self.children[1].bestAttribute )
            
            return ans
            
    def PrintTree(self, columnNames ):
        ans = []
        ans = self.PrintRec( columnNames, ans, self.bestAttribute)
        answer = ""
        loop = 0
        dic_loop = {}
        
        for i in ans:
            temp = True
            if i == "0":
                answer+=i
                temp = False
            if i == "1":
                answer+=i
                temp = False
            
            if temp:
                if dic_loop.get(i[0:2]) == None:
                    dic_loop[ i[0:2] ] = loop
                else:
                    loop = dic_loop.get(i[0:2])

                answer += "\n"
                for j in range(loop):
                    answer+= "| "
                answer += i
            loop +=1
        print(answer)

In [10]:
def CalculateAccuracy( dt, X, Y):
    predict_Y = np.zeros( ( Y.shape ) )
    for i in range( len(X) ):
        predict_Y[i] = dt.Evaluate( X[i,:] )

    no_of_matches = np.sum( predict_Y == Y)
    accuracy = no_of_matches/len(Y)
    
    #Accuracy Calculation
    return accuracy

Purning Algorithm by Classification Performance on Validation Set

In [11]:
# Count leaves in a tree
def CountLeaves(dt):
    if dt.isLeaf:
        return 1
    else:
        n = 0
        for val,child in dt.children.items():
            n += CountLeaves(child)
    return n

# Check if a node is a twig
def isTwig(dt):
    for val,child in dt.children.items():
        if not child.isLeaf:
            return False
    return True

# First create an empty list of error counts at nodes
def CreateNodeList( dt, nodeError={} ):
    nodeError[dt] = 0
    for val,child in dt.children.items():
        CreateNodeList(child,nodeError)
    return nodeError

# Pass a single instance down the tree and note node errors
def ClassifyValidationDataInstance( dt, validationDataInstance, nodeError ):
    labels = validationDataInstance[len(validationDataInstance)-1]
    if dt.majorityClass != labels:
        nodeError[dt] += 1
    if not dt.isLeaf:
        childNode = dt.children[ validationDataInstance[dt.bestAttribute] ]
        ClassifyValidationDataInstance( childNode, validationDataInstance, nodeError )
    return

# Count total node errors for validation data
def ClassifyValidationData(dt, validationData):
    nodeErrorCounts = CreateNodeList(dt)
    for instance in validationData:
        ClassifyValidationDataInstance(dt, instance, nodeErrorCounts)
    return nodeErrorCounts

# Second pass:  Create a heap with twigs using nodeErrorCounts
def CollectTwigsByErrorCount(dt, nodeErrorCounts, heap=[]):
    if isTwig(dt):
        # Count how much the error would increase if the twig were trimmed
        twigErrorIncrease = nodeErrorCounts[dt]
        for val,child in dt.children.items():
            twigErrorIncrease -= nodeErrorCounts[child]
        heap.append([twigErrorIncrease, dt])
    else:
        for val,child in dt.children.items():
            CollectTwigsByErrorCount(child, nodeErrorCounts, heap)
    return heap

# Third pass: Prune a tree to have nLeaves leaves
# Assuming heappop pops smallest value
def PruneByClassificationError(dt, validationData, nLeaves = -1):
    # First obtain error counts for validation data
    nodeErrorCounts = ClassifyValidationData(dt, validationData)
    
    # Get Twig Heap
    twigHeap = CollectTwigsByErrorCount(dt, nodeErrorCounts)

    totalLeaves = CountLeaves(dt)
    while totalLeaves > nLeaves:

        #Find index of minimum value of Twig from the Heap
        minVal = math.inf
        twigHeap_index = -1
        loop = 0
        for x in twigHeap:
            if x[0] < minVal:
                twigHeap_index = loop
            loop+=1
        
        if twigHeap[twigHeap_index][0] > 0 and nLeaves == -1:
            break
        
        twig = twigHeap[twigHeap_index][1]
        twigHeap.pop(twigHeap_index)
        totalLeaves -= (len(twig.children) - 1) 
        parent = twig.parent
        
        #Removing Twig from the original tree
        dt.RemoveChild( twig )

        # Check if the parent is a twig and, if so, put it in the heap
        if isTwig(parent):
            twigErrorIncrease = nodeErrorCounts[parent]
            for val,child in parent.children.items():
                twigErrorIncrease -= nodeErrorCounts[child]
            twigHeap.append([twigErrorIncrease, parent])
    
    return dt

In [12]:
dt = DecisionTree( ds1_train_X, ds1_train_Y, "VarianceImpurity"  )

print( "Before Pruning:--- ")
dt.PrintTree( ds1_train.columns )
print( "Accuracy on Validation Data :- " + str(CalculateAccuracy( dt, ds1_val_X, ds1_val_Y ) ) )
print( "Accuracy on Testing Data :-- " + str(CalculateAccuracy( dt, ds1_test_X, ds1_test_Y ) ) )

dt = PruneByClassificationError( dt, np.concatenate((ds1_val_X,ds1_val_Y),axis=1) )

print( "\nAfter Pruning:--- ")
dt.PrintTree( ds1_train.columns )
print( "Accuracy on Validation Data :- " + str(CalculateAccuracy( dt, ds1_val_X, ds1_val_Y ) ) )
print( "Accuracy on Testing Data :-- " + str(CalculateAccuracy( dt, ds1_test_X, ds1_test_Y ) ) )

Before Pruning:--- 

XI = 0 : 
| XG = 0 : 0
| XG = 1 : 
| | XS = 0 : 
| | | XJ = 0 : 0
| | | XJ = 1 : 
| | | | XQ = 0 : 
| | | | | XE = 0 : 0
| | | | | XE = 1 : 1
| | | | XQ = 1 : 0
| | XS = 1 : 0
XI = 1 : 
| XH = 0 : 
| | | | | XE = 0 : 0
| | | | | XE = 1 : 
| | | | | | XL = 0 : 
| | | XJ = 0 : 0
| | | XJ = 1 : 0
| | | | | | XL = 1 : 0
| XH = 1 : 
| | XC = 0 : 
| | | XN = 0 : 
| | | | XD = 0 : 
| | | | | XF = 0 : 0
| | | | | XF = 1 : 
| | | | | | XU = 0 : 
| | | | | | | XB = 0 : 1
| | | | | | | XB = 1 : 1
| | | | | | XU = 1 : 1
| | | | XD = 1 : 
| | | | | XP = 0 : 0
| | | | | XP = 1 : 
| | XS = 0 : 0
| | XS = 1 : 1
| | | XN = 1 : 
| | | | | | | XB = 0 : 
| | | | | | | | XR = 0 : 
| | | | | XE = 0 : 
| XG = 0 : 1
| XG = 1 : 0
| | | | | XE = 1 : 
| | | XJ = 0 : 1
| | | XJ = 1 : 
| | | | | XP = 0 : 1
| | | | | XP = 1 : 1
| | | | | | | | XR = 1 : 
| | | | | | XU = 0 : 0
| | | | | | XU = 1 : 0
| | | | | | | XB = 1 : 
| | | | | XF = 0 : 
| XG = 0 : 1
| XG = 1 : 1
| | | | | XF = 1 : 
| | | |

In [13]:
dt = DecisionTree( ds2_train_X, ds2_train_Y, "VarianceImpurity"  )

print( "Before Pruning:--- ")
#dt.PrintTree( ds2_train.columns )
print( "Accuracy on Validation Data :- " + str(CalculateAccuracy( dt, ds2_val_X, ds2_val_Y ) ) )
print( "Accuracy on Testing Data :-- " + str(CalculateAccuracy( dt, ds2_test_X, ds2_test_Y ) ) )

dt = PruneByClassificationError( dt, np.concatenate((ds2_val_X,ds2_val_Y),axis=1) )

print( "\nAfter Pruning:--- ")
#dt.PrintTree( ds2_train.columns )
print( "Accuracy on Validation Data :- " + str(CalculateAccuracy( dt, ds2_val_X, ds2_val_Y ) ) )
print( "Accuracy on Testing Data :-- " + str(CalculateAccuracy( dt, ds2_test_X, ds2_test_Y ) ) )

Before Pruning:--- 
Accuracy on Validation Data :- 0.6616666666666666
Accuracy on Testing Data :-- 0.6383333333333333

After Pruning:--- 
Accuracy on Validation Data :- 0.67
Accuracy on Testing Data :-- 0.6333333333333333


In [14]:
dt = DecisionTree( ds1_train_X, ds1_train_Y, "GainRatio"  )
print( "Before Pruning:--- ")
#dt.PrintTree( ds1_train.columns )
print( "Accuracy on Validation Data :- " + str(CalculateAccuracy( dt, ds1_val_X, ds1_val_Y ) ) )
print( "Accuracy on Testing Data :-- " + str(CalculateAccuracy( dt, ds1_test_X, ds1_test_Y ) ) )

dt = PruneByClassificationError( dt, np.concatenate((ds1_val_X,ds1_val_Y),axis=1) )

print( "\nAfter Pruning:--- ")
#dt.PrintTree( ds1_train.columns )
print( "Accuracy on Validation Data :- " + str(CalculateAccuracy( dt, ds1_val_X, ds1_val_Y ) ) )
print( "Accuracy on Testing Data :-- " + str(CalculateAccuracy( dt, ds1_test_X, ds1_test_Y ) ) )

Before Pruning:--- 
Accuracy on Validation Data :- 0.965
Accuracy on Testing Data :-- 0.9566666666666667

After Pruning:--- 
Accuracy on Validation Data :- 0.965
Accuracy on Testing Data :-- 0.9566666666666667


In [15]:
dt = DecisionTree( ds2_train_X, ds2_train_Y, "GainRatio"  )

print( "Before Pruning:--- ")
#dt.PrintTree( ds2_train.columns )
print( "Accuracy on Validation Data :- " + str(CalculateAccuracy( dt, ds2_val_X, ds2_val_Y ) ) )
print( "Accuracy on Testing Data :-- " + str(CalculateAccuracy( dt, ds2_test_X, ds2_test_Y ) ) )

dt = PruneByClassificationError( dt, np.concatenate((ds2_val_X,ds2_val_Y),axis=1) )

print( "\nAfter Pruning:--- ")
#dt.PrintTree( ds2_train.columns )
print( "Accuracy on Validation Data :- " + str(CalculateAccuracy( dt, ds2_val_X, ds2_val_Y ) ) )
print( "Accuracy on Testing Data :-- " + str(CalculateAccuracy( dt, ds2_test_X, ds2_test_Y ) ) )

Before Pruning:--- 
Accuracy on Validation Data :- 0.75
Accuracy on Testing Data :-- 0.7266666666666667

After Pruning:--- 
Accuracy on Validation Data :- 0.7583333333333333
Accuracy on Testing Data :-- 0.7316666666666667
